## This file implements translation evaluations using a modified GEMBA prompt system for the train dataset. Due to OpenAI rate limits, the evaluations are run with chunks of the training set producing split results.

### Install Libraries

In [1]:
%pip install pandas
%pip install langchain
%pip install langchain-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Imports

In [2]:
import os
import ast
import pandas as pd
from tqdm import tqdm

# Langchain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

### Globals

In [3]:
LLM_MODEL_NAME="gpt-3.5-turbo"

### Utility function to set up the OpenAI API

In [4]:
def load_openai_key():
    try:
        # Open and read the entire content of the file
        with open("openai-key.txt", 'r') as file:
            contents = file.read()
        
        return contents
    
    except FileNotFoundError:
        print(f"The file does not exist.")
        return None
    except Exception as e:
        # Handle other potential exceptions (e.g., permission errors)
        print(f"An error occurred while reading the file: {str(e)}")
        return None

openai_key = load_openai_key()
os.environ["OPENAI_API_KEY"] = openai_key

chat = ChatOpenAI(model="gpt-3.5-turbo")

### Prompt definition

In [5]:
system = """\
You are a helpful evaluator of the quality of translations.
Score the following translations from English to Portuguese on a continuous scale from 0 to 100, 
where a score of zero means "no meaning preserved" and score of one hundred means
"perfect meaning and grammar".
"""

human = """\
En: {source_seg}
Pt 1: "{target_seg1}" 
Pt 2: "{target_seg2}"
Pt 3: "{target_seg3}"
Reply with only the number scores of your evaluation, in a python list:
"""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

### Initial test

In [6]:
chain = prompt | chat
response = chain.invoke(input={"source_seg": "There are more things between heaven and earth",
                    "target_seg1": "Existem mais coisas entre o céu e a terra",
                    "target_seg2": "Existem coisas entre o céu e a terra",
                    "target_seg3": "Existem coisas entre o paraíso e a terra"})

In [7]:
print(response.content)

[95, 80, 60]


### Load all source and target language datasets

#### Source sets

In [8]:
folder_path = 'data/train_sets'
num_chunks = len(os.listdir(folder_path))

df_train_sets = []
for i in range(num_chunks):
    file_path = folder_path + '/train_' + str(i+1) + '.csv'
    df = pd.read_csv(file_path)
    df = df.drop(columns=['Unnamed: 0'])
    df_train_sets.append(df)

In [9]:
df_train_sets[0]

,context,question,answerA,answerB,answerC,correct
0,"Even though she had homework to do that night,...",What will Jesse want to do next?,read homework to Skylar,help Skylar finish,skip her studying,B
1,"After school, Casey met the friend at a bar so...",Why did Casey do this?,have a good idea of the material,goof around with a friend,have a few drinks and leave,A
2,Jesse went quickly to their mother and their m...,How would Jesse feel afterwards?,wasting their time,that they are a good child,that their mother always calls them,B
3,Robin knew that Kai really wanted her to the l...,Why did Robin do this?,paid her to say that she liked it,she never really liked Kai or her fashion,she knew Kai wanted Robin to like the outfit,C
4,Addison slept well last night after playing ba...,Why did Addison do this?,regain her energy,hit a home run,run the bases,A
...,...,...,...,...,...,...
4995,Bailey scarred the hell out of Alex because Al...,What will happen to Alex?,try to beat Alex up,have no friend,get a restraining order against Alex,B
4996,Quinn drank a lot of alcohol while they were o...,How would Quinn feel afterwards?,a hardcore party person,sober,hungover,C
4997,Carson sent Bailey's lunch to school after he ...,What will Carson want to do next?,remember bailey's lunch,make sure the lunch gets there,say sorry multiple times,B
4998,Jan decided to watch the news today because a ...,Why did Jan watch the news?,watched because the news was being attacked,watched because a war began last night,watched because her town was broke,B


#### Target sets

In [10]:
folder_path = 'translated/train_sets'
num_chunks = len(os.listdir(folder_path))

df_train_pt_sets = []
for i in range(num_chunks):
    file_path = folder_path + '/train_pt_' + str(i+1) + '.csv'
    df = pd.read_csv(file_path)
    df = df.drop(columns=['Unnamed: 0'])
    df_train_pt_sets.append(df)

In [11]:
df_train_pt_sets[0]

,context_1,context_2,context_3,question_1,question_2,question_3,answerA_1,answerA_2,answerA_3,answerB_1,answerB_2,answerB_3,answerC_1,answerC_2,answerC_3,correct
0,Mesmo que ela tivesse trabalho de casa para fa...,Embora tivesse realizado trabalho doméstico pa...,Mesmo tendo de fazer o dever de casa naquela n...,O que Jesse vai querer fazer a seguir?,O que Jesse quer fazer em seguida?,O que é que o Jesse vai querer fazer a seguir?,Leia lição de casa para Skylar,Leia o serviço de casa para Skylar,Leia os trabalhos de casa à Skylar.,Ajuda Skylar finish,Ajuda de acabamento Skylar,Ajudem a Skylar a terminar .,ignorá-la estudando,skip seus estudos,Não a deixes estudar.,B
1,"Depois da escola, Casey conheceu o amigo em um...","Após a escola, Casey conheceu o amigo em um ba...","Depois da escola, o Casey conheceu a amiga num...",Porque é que o Casey fez isto?,Por que o Casey?,Porque fez o Casey isto?,Tenha uma boa ideia do material,têm uma boa ideia do material,ter uma boa ideia do material,goof em torno de com um amigo,náuseas,brincar com um amigo,Beba alguns drinques e vá embora,tem algumas bebidas e folhas.,Bebam um copo e vão-se embora .,A
2,Jesse foi rapidamente para sua mãe e sua mãe d...,Jesse chegou rapidamente à mãe e a mãe lhes di...,Jesse foi rapidamente para a mãe deles e a mãe...,Como Jesse se sentiria depois?,Como Jesse se sentiria depois?,Como se sentiria o Jesse depois?,Desperdiçando o seu tempo,desperdícios,a perder o seu tempo.,que eles são uma boa criança,que são uma criança boa,Que são uma boa criança.,que sua mãe sempre os chama,que a mãe sempre os chama,que a mãe sempre os chama,B
3,Robin sabia que Kai realmente queria que ela g...,Robin sabia que Kai realmente queria que ela f...,A Robin sabia que o Kai queria que ela gostass...,Porque é que a Robin fez isto?,Por que Robin fez isso?,Porque é que a Robin fez isto?,Pagou-lhe para dizer que ela gostou,pagou-lhe que ela gostava de,Pagou-lhe para dizer que gostava.,ela nunca gostou muito de Kai ou sua moda,nunca gostava mesmo de Kai ou de sua moda,Ela nunca gostou muito do Kai ou da moda dela .,Ela sabia que Kai queria que Robin gostasse da...,ela sabia que Kai queria Robin gostar do outfit,Ela sabia que o Kai queria que a Robin gostass...,C
4,A Addison dormiu bem ontem à noite depois de j...,O adison dormiu bem na noite seguinte ao jogo ...,Addison dormiu bem ontem à noite depois de jog...,Porque é que a Addison fez isto?,Por que o Addison fez isso?,Porque é que a Addison fez isto?,Recupere sua energia,recuperar sua energia,Recupera a sua energia.,Atingir um home run,batimentos em casa,bateu um home run,Executar as bases,Executar as bases,- Não .,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Bailey tirou as cicatrizes de Alex porque Alex...,Bailey cicatriou o inferno de Alex porque Alex...,A Bailey deixou cicatrizes no Alex porque ele ...,O que vai acontecer com Alex?,O que acontecerá com Alex?,O que vai acontecer com o Alex?,tentar bater Alex para cima,tentar vencer Alex,Tentei bater no Alex .,não tem amigo,Não tem amigos,"Não tenho amigo,",obter uma ordem de restrição contra Alex,obter uma ordem de retenção contra Alex,Obter uma ordem de restrição contra o Alex,B
4996,Quinn bebeu muito álcool enquanto eles estavam...,Quinn bebe muito álcool enquanto eles estavam ...,Quinn bebeu muito álcool enquanto estavam numa...,Como Quinn se sentiria depois?,Como se sentiria Quinn depois?,Como se sentiria o Quinn depois?,uma pessoa de festa hardcore,uma pessoa emblemática,Uma pessoa de festas hardcore,sóbria,nã,sóbrio,ressaca,náusea,ressaca,C
4997,Carson mandou o almoço de Bailey para a escola...,Carson enviou o almoço de Bailey para a escola...,O Carson mandou o almoço do Bailey para a esco...,O que Carson vai querer fazer a seguir?,O que queremos por Carson?,O que é que o Carson vai querer fazer a seguir?,Lembra-te do almoço do Bailey,lembrar o almoço do baile,Lembras-te do almoço da Bailey?,Certifique-se de que o almoço chegue lá,esqueça o almoç

### Functions for evaluation

In [12]:
def create_sequences(i, df_src_lang, df_tgt_lang):

        sentence_list = ['context', 'question', 'answerA', 'answerB', 'answerC']
        english_str=""
        for sentence in sentence_list:
                english_str += str(df_src_lang.loc[i, sentence])
                english_str += "\n"

        portuguese_1=""
        sentence_list = ['context_1', 'question_1', 'answerA_1', 'answerB_1', 'answerC_1']
        for sentence in sentence_list:
                portuguese_1 += str(df_tgt_lang.loc[i, sentence])
                portuguese_1 += "\n"

        portuguese_2=""
        sentence_list = ['context_2', 'question_2', 'answerA_2', 'answerB_2', 'answerC_2']
        for sentence in sentence_list:
                portuguese_2 += str(df_tgt_lang.loc[i, sentence])
                portuguese_2 += "\n"

        portuguese_3=""
        sentence_list = ['context_3', 'question_3', 'answerA_3', 'answerB_3', 'answerC_3']
        for sentence in sentence_list:
                portuguese_3 += str(df_tgt_lang.loc[i, sentence])
                portuguese_3 += "\n"

        return english_str, portuguese_1, portuguese_2, portuguese_3

In [13]:
def get_best_translations(df_src_lang, df_tgt_lang):

    total_tokens = 0

    df_output = df_src_lang.copy()
    df_output.loc[:, ['context', 'question', 'answerA', 'answerB', 'answerC']] = None
    df_rankings = pd.DataFrame(columns=['MarianMT', 'Unicamp_T5', 'NLLB_1.3b'])

    for i in tqdm(range(len(df_src_lang))):

        # OpenAI call
        chain = prompt | chat
        english, translation_pt1, translation_pt2, translation_pt3 = create_sequences(i,
                                                                                      df_src_lang,
                                                                                      df_tgt_lang)
        response = chain.invoke(input={"source_seg": english,
                        "target_seg1": translation_pt1,
                        "target_seg2": translation_pt2,
                        "target_seg3": translation_pt3})
        
        total_tokens += response.response_metadata.get('token_usage').get('total_tokens')

        results = ast.literal_eval(response.content)            
        best = results.index(max(results))
        
        # Add to rankings dataset
        df_rankings.loc[len(df_rankings)] = results
        
        # Copy to final dataset
        col_list = ['context', 'question', 'answerA', 'answerB', 'answerC']
        for col in col_list:
            col_best = col + '_' + str(best+1)
            best_result = df_tgt_lang.loc[i, col_best]
            sentence_idx = col_list.index(col)
            df_output.iat[i, sentence_idx] = best_result

    print(f"Total tokens used: {total_tokens}")
    
    return df_output, df_rankings

### Training Set 1

In [14]:
df_train_1_final, df_train_1_rankings = get_best_translations(df_train_sets[0], df_train_pt_sets[0])

100%|██████████| 5000/5000 [52:25<00:00,  1.59it/s]  

Total tokens used: 1539727


In [17]:
df_train_1_final.to_csv('translated/final/train_sets/train_1.csv')
df_train_1_rankings.to_csv('rankings/train_sets/train_1_rankings.csv')

### Training Set 2

In [14]:
df_train_2_final, df_train_2_rankings = get_best_translations(df_train_sets[1], df_train_pt_sets[1])

 21%|██        | 1055/5000 [33:22<2:04:48,  1.90s/it] 


KeyboardInterrupt: 

In [ ]:
df_train_2_final.to_csv('translated/final/train_sets/train_2.csv')
df_train_2_rankings.to_csv('rankings/train_sets/train_2_rankings.csv')

### Training Set 3

In [ ]:
df_train_3_final, df_train_3_rankings = get_best_translations(df_train_sets[2], df_train_pt_sets[2])

In [ ]:
df_train_3_final.to_csv('translated/final/train_sets/train_3.csv')
df_train_3_rankings.to_csv('rankings/train_sets/train_3_rankings.csv')